# 프롬프트(시스템 역할 메시지)를 통한 챗봇 구현

## OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
import gradio as gr

# .env 로드 및 API 키 확인
load_dotenv()

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

## 초기 메시지 - 시스템 역할 메시지 구성

In [ ]:
# 초기 시스템 메시지
initial_messages = [
    SystemMessage(content="당신은 '명탐정 코난' 전문가입니다. 질문에 친절하고 정확하게 답해 주세요.")
]

## 채팅 처리 함수
Gradio UI와 연동하는 인터페이스 함수
* `convert_history_to_display` : 채팅 기록을 Gradio UI 출력 형식으로 변환하는 함수
    - 사용자가 입력한 정보(`HumanMessage`)와 LLM의 응답(`AIMessage`)인 경우 채팅 출력(`chat_display`)에 추가 
    - 초기 메시지에 설정된 시스템 메시지(`SystemMessage`)는 출력되지 않음

* `chat_fn` : UI 채팅 처리 함수
    - UI 입력창에서 엔터 이벤트(`txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])`)가 발생 하면 호출됨 
    - 채팅 출력(`chat_display`)의 포맷(`{"role": "...", "content": "..."}`)에서 `role`은 **'user'** 와 **'assistant'**'로 지정되어야함.

* `yield`
    - Python에서 함수를 특별한 종류의 이터레이터(iterator)인 제너레이터(generator)로 만들어주는 키워드
    - 일반적인 함수가 return으로 값을 반환하면서 종료되는 반면, yield를 사용한 함수는 값을 해당 시점에 값을 반환 후 상태를 유지한 채 잠시 멈췄다가, 다음 요청이 오면 멈춘 지점부터 실행을 재개
    - 모든 결과를 한 번에 계산해서 메모리에 저장하는 대신, 필요한 시점에 값을 하나씩 생성 - **지연 평가(Lazy Evaluation)**
    - `txt.submit` 이벤트 리스너에 의해 제너레이터의 다음 순서를 실행(`next()`가 호출)하게 됨.

In [ ]:
# 채팅 기록을 UI 출력 형식으로 변환하는 함수
def convert_history_to_display(history):
    chat_display = []
    for msg in history:
        if isinstance(msg, HumanMessage):
            chat_display.append({"role": "user", "content": msg.content})
        elif isinstance(msg, AIMessage):
            chat_display.append({"role": "assistant", "content": msg.content})
    return chat_display

# 채팅 처리 함수
def chat_fn(user_input, chat_history):
    human_message = HumanMessage(content=user_input)
    chat_history.append(human_message)

    working_display = convert_history_to_display(chat_history)
    working_display.append({"role": "assistant", "content": "답변을 생성 중입니다... 🤔"})
    yield working_display, chat_history

    ai_message = llm.invoke(chat_history)
    chat_history.append(ai_message)

    # 5. 최종 채팅 기록으로 UI출력
    final_display = convert_history_to_display(chat_history)
    yield final_display, chat_history

## Gradio 인터페이스 구성

패키지 설치 `pip install gradio`

Gradio의 Blocks UI 구성 방식을 사용해 명탐정 코난 퀴즈 챗봇 인터페이스를 구성
- `with gr.Blocks() as demo `: Gradio의 레이아웃 기반 UI 블록을 시작 - 앱 전체 인스턴스를 `demo`로 선언
- `gr.Markdown(...)` : 화면 상단에 표시할 설명 문구 (Markdown 문법 사용)
- `gr.Chatbot(...)` : 사용자와 AI 간의 대화를 보여주는 대화창 컴포넌트
- `gr.Textbox(...)` : 사용자가 입력할 수 있는 텍스트 입력창 (placeholder로 입력 힌트를 표시)
- `gr.State(initial_messages.copy())` : `initial_messages.copy()`를 호출하여 초기 상태를 저장

※ 입력 처리 상세 설명

1. `txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])`
    - `txt.submit(...)` : 사용자가 텍스트박스에 입력하고 Enter를 누르면 실행
    - `chat_fn` : 메인 처리 함수 - 사용자의 입력을 받아 챗봇 응답과 상태를 계산 또는 
    - `inputs=[txt, state]` : 텍스트 입력값과 이전 상태(state)를 chat_fn에 전달
    - `outputs=[chatbot, state]` : 함수 결과로 나온 대화 내용과 상태를 챗봇 창과 내부 상태에 각각 반영

2. `txt.submit(lambda: "", None, txt)`
    - `lambda: ""` : 아무 동작 없이 빈 문자열을 반환하는 함수
    - `None` : 입력값은 없음
    - `txt` : 출력 대상은 텍스트박스 자신 - 입력 후 자동으로 입력창을 초기화

※ 람다(`lambda`) 함수는 이름 없이 간단하게 사용하는 **한 줄짜리 익명 함수(anonymous function)** - 임시로 사용할 간단한 기능이 필요할 때 구현

```python
lambda 매개변수1, 매개변수2, ... : 표현식

# 짝수만 걸러내고 싶을 때
numbers = [1, 2, 3, 4, 5, 6]
even_numbers = list(filter(lambda x: x % 2 == 0, numbers)) # [2, 4, 6]
```

In [ ]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️‍♂️ 명탐정 코난 전문가에게 무엇이든 물어보세요!")

    chatbot = gr.Chatbot(height=300, type="messages", label="명탐정 코난 전문가 챗봇")
    txt = gr.Textbox(placeholder="질문을 입력하세요...", show_label=False)
    state = gr.State(initial_messages.copy())

    txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])
    txt.submit(lambda: "", None, txt)  # 입력창 초기화

demo.launch()

-----
** End of Documents **